In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
spark = (
    SparkSession.builder.appName("Read S3 File")
    .master("spark://localhost:7077")
    .config(
        "spark.jars",
        "/opt/bitnami/spark/jars/hadoop-aws-3.3.4.jar,/opt/bitnami/spark/jars/aws-java-sdk-bundle-1.12.262.jar",
    )
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/12/03 22:07:46 WARN Utils: Your hostname, cpPC resolves to a loopback address: 127.0.1.1; using 172.25.249.11 instead (on interface eth0)
23/12/03 22:07:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/03 22:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")

In [5]:
df = spark.range(1000).toDF("number")
df.show(2)

+------+
|number|
+------+
|     0|
|     1|
+------+
only showing top 2 rows



In [13]:
# Read
taxi_zone_df = (
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv("s3a://de-666ne92p/tmp/taxi+_zone_lookup.csv")
    .cache()
)

In [14]:
taxi_zone_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [15]:
date_dim_df = (
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv("s3a://de-666ne92p/tmp/date_dim.csv")
    .cache()
)

In [16]:
date_dim_df.show(2)

+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|date_key|      date|day_of_week|day_name|day_of_month|day_of_year|week_of_year|month|month_name|quarter|year|is_holiday|is_weekend|is_workday|
+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|   10957|2000-01-01|          6|Saturday|          31|          1|          52|    1|   January|      1|2000|      true|      true|     false|
|   10958|2000-01-02|          0|  Sunday|          31|          2|          52|    1|   January|      1|2000|     false|      true|     false|
+--------+----------+-----------+--------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
only showing top 2 rows



In [21]:
def dimention_df():
    def taxi_zone_df():
        return (
            spark.read.option("inferSchema", "true")
            .option("header", "true")
            .csv("s3a://de-666ne92p/tmp/taxi+_zone_lookup.csv")
            .cache()
        )

    def date_dim_df():
        return (
            spark.read.option("inferSchema", "true")
            .option("header", "true")
            .csv("s3a://de-666ne92p/tmp/date_dim.csv")
            .cache()
        )

    def vendor_df():
        schema = StructType(
            [
                StructField("VENDOR_KEY", IntegerType(), True),
                StructField("VENDOR_ID", IntegerType(), True),
                StructField("VENDOR_NAME", StringType(), True),
            ]
        )

        # Your CSV-like data
        data = [(1, 1, "Creative Mobile Technologies, LLC"),
                (2, 2, "VeriFone Inc.")]

        # Create a DataFrame
        return spark.createDataFrame(data, schema=schema)

    def trip_type_df():
        schema = StructType(
            [
                StructField("TRIP_TYPE_KEY", IntegerType(), True),
                StructField("TRIP_TYPE_ID", IntegerType(), True),
                StructField("TRIP_TYPE_NAME", StringType(), True),
            ]
        )

        data = [(1, 1, "Street-hail"), (2, 2, "Dispatch")]

        return spark.createDataFrame(data, schema=schema)

    def rate_code_df():
        schema = StructType(
            [
                StructField("RATE_CODE_KEY", IntegerType(), True),
                StructField("RATE_CODE_ID", IntegerType(), True),
                StructField("RATE_CODE_EFFECT", StringType(), True),
            ]
        )

        data = [
            (1, 1, "Standard rate"),
            (2, 2, "JFK"),
            (3, 3, "Newark"),
            (4, 4, "Nassau or Westchester"),
            (5, 5, "Negotiated fare"),
            (6, 6, "Group ride"),
        ]

        return spark.createDataFrame(data, schema=schema)

    def payment_type_df():
        schema = StructType(
            [
                StructField("PAYMENT_TYPE_KEY", IntegerType(), True),
                StructField("PAYMENT_TYPE_ID", IntegerType(), True),
                StructField("PAYMENT_TYPE_NAME", StringType(), True),
            ]
        )

        data = [
            (1, 1, "Credit card"),
            (2, 2, "Cash"),
            (3, 3, "No charge"),
            (4, 4, "Dispute"),
            (5, 5, "Unknown"),
            (6, 6, "Voided trip"),
        ]

        return spark.createDataFrame(data, schema=schema)

    return taxi_zone_df(), date_dim_df(), vendor_df(), trip_type_df(), rate_code_df(), payment_type_df()

In [22]:
(
    taxi_zone_df,
    date_dim_df,
    vendor_df,
    trip_type_df,
    rate_code_df,
    payment_type_df,
) = dimention_df()

taxi_zone_df.show(5)

23/12/03 22:32:31 WARN CacheManager: Asked to cache already cached data.        


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



23/12/03 22:32:37 WARN CacheManager: Asked to cache already cached data.        


In [23]:
date_dim_df.show(5)

+--------+----------+-----------+---------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|date_key|      date|day_of_week| day_name|day_of_month|day_of_year|week_of_year|month|month_name|quarter|year|is_holiday|is_weekend|is_workday|
+--------+----------+-----------+---------+------------+-----------+------------+-----+----------+-------+----+----------+----------+----------+
|   10957|2000-01-01|          6| Saturday|          31|          1|          52|    1|   January|      1|2000|      true|      true|     false|
|   10958|2000-01-02|          0|   Sunday|          31|          2|          52|    1|   January|      1|2000|     false|      true|     false|
|   10959|2000-01-03|          1|   Monday|          31|          3|           1|    1|   January|      1|2000|     false|     false|      true|
|   10960|2000-01-04|          2|  Tuesday|          31|          4|           1|    1|   January|      1|2000|     false|     fal

In [24]:
vendor_df.show(5)

+----------+---------+--------------------+
|VENDOR_KEY|VENDOR_ID|         VENDOR_NAME|
+----------+---------+--------------------+
|         1|        1|Creative Mobile T...|
|         2|        2|       VeriFone Inc.|
+----------+---------+--------------------+



In [25]:
trip_type_df.show(5)

+-------------+------------+--------------+
|TRIP_TYPE_KEY|TRIP_TYPE_ID|TRIP_TYPE_NAME|
+-------------+------------+--------------+
|            1|           1|   Street-hail|
|            2|           2|      Dispatch|
+-------------+------------+--------------+



In [26]:
rate_code_df.show(5)

+-------------+------------+--------------------+
|RATE_CODE_KEY|RATE_CODE_ID|    RATE_CODE_EFFECT|
+-------------+------------+--------------------+
|            1|           1|       Standard rate|
|            2|           2|                 JFK|
|            3|           3|              Newark|
|            4|           4|Nassau or Westche...|
|            5|           5|     Negotiated fare|
+-------------+------------+--------------------+
only showing top 5 rows



In [27]:
payment_type_df.show(5)

+----------------+---------------+-----------------+
|PAYMENT_TYPE_KEY|PAYMENT_TYPE_ID|PAYMENT_TYPE_NAME|
+----------------+---------------+-----------------+
|               1|              1|      Credit card|
|               2|              2|             Cash|
|               3|              3|        No charge|
|               4|              4|          Dispute|
|               5|              5|          Unknown|
+----------------+---------------+-----------------+
only showing top 5 rows

